# Define models

In [ ]:
#| default_exp models

In [ ]:
#| exports
from functools import partial
from math import log2

import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from timm.models.vision_transformer import Block, PatchEmbed

from advanced_autoencoders.utils import get_2d_sincos_pos_embed

In [ ]:
#| exports
class ResBlock(nn.Module):
    def __init__(self, chan):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(chan, chan, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(chan, chan, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(chan, chan, 1),
        )

    def forward(self, x):
        return self.net(x) + x

In [ ]:
#| exports
class VectorQuantize(nn.Module):
    def __init__(self, dim, n_embed, commitment=1.0):
        super().__init__()

        self.dim = dim
        self.n_embed = n_embed
        self.commitment = commitment

        self.embed = nn.Embedding(self.n_embed, self.dim)
        self.embed.weight.data.uniform_(-1 / self.n_embed, 1 / self.n_embed)

    def forward(self, input):
        dtype = input.dtype  # noqa
        input_shape = input.shape
        flatten = input.reshape(-1, self.dim)

        distances = torch.cdist(flatten, self.embed.weight)
        encoding_index = torch.argmin(distances, dim=1)

        quantized = torch.index_select(self.embed.weight, 0, encoding_index).view(
            input_shape
        )

        return quantized

In [ ]:
#| exports
class MyVQVAE(nn.Module):
    def __init__(
        self,
        image_size=256,
        num_tokens=8192,
        codebook_dim=8,
        num_layers=4,
        num_resnet_blocks=2,
        hidden_dim=64,
        channels=3,
        smooth_l1_loss=False,
        # vq_decay = 0.8,
        commitment_weight=0.25,
        use_vq_commit_loss=True,
    ):
        super().__init__()
        assert log2(image_size).is_integer(), "image size must be a power of 2"
        assert num_layers >= 1, "number of layers must be greater than or equal to 1"
        has_resblocks = num_resnet_blocks > 0

        self.image_size = image_size
        self.num_tokens = num_tokens
        self.num_layers = num_layers
        self.commitment_weight = commitment_weight
        self.use_vq_commit_loss = use_vq_commit_loss

        self.vq = VectorQuantize(
            dim=codebook_dim,
            n_embed=num_tokens,
            # decay = vq_decay,
            commitment=commitment_weight,
        )

        hdim = hidden_dim  # noqa

        enc_chans = [hidden_dim] * num_layers
        dec_chans = list(reversed(enc_chans))

        enc_chans = [channels, *enc_chans]

        dec_init_chan = codebook_dim if not has_resblocks else dec_chans[0]
        dec_chans = [dec_init_chan, *dec_chans]

        enc_chans_io, dec_chans_io = map(
            lambda t: list(zip(t[:-1], t[1:])), (enc_chans, dec_chans)
        )

        enc_layers = []
        dec_layers = []

        for (enc_in, enc_out), (dec_in, dec_out) in zip(enc_chans_io, dec_chans_io):
            enc_layers.append(
                nn.Sequential(
                    nn.Conv2d(enc_in, enc_out, 4, stride=2, padding=1), nn.ReLU()
                )
            )
            dec_layers.append(
                nn.Sequential(
                    nn.ConvTranspose2d(dec_in, dec_out, 4, stride=2, padding=1),
                    nn.ReLU(),
                )
            )

        for _ in range(num_resnet_blocks):
            dec_layers.insert(0, ResBlock(dec_chans[1]))
            enc_layers.append(ResBlock(enc_chans[-1]))

        if num_resnet_blocks > 0:
            dec_layers.insert(0, nn.Conv2d(codebook_dim, dec_chans[1], 1))

        enc_layers.append(nn.Conv2d(enc_chans[-1], codebook_dim, 1))
        dec_layers.append(nn.Conv2d(dec_chans[-1], channels, 1))

        self.encoder = nn.Sequential(*enc_layers)
        self.decoder = nn.Sequential(*dec_layers)

        self.loss_fn = F.smooth_l1_loss if smooth_l1_loss else F.mse_loss

    def get_vq_commitment_loss(self, quantized, inputs):
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        c_loss = q_latent_loss + self.commitment_weight * e_latent_loss

        quantized = inputs + (quantized - inputs).detach()

        return quantized, c_loss

    def forward(
        self,
        img,
        return_loss=False,
        return_recons=False,
        return_encoded=False,
    ):
        shape, device = img.shape, img.device  # noqa

        encoded = self.encoder(img)

        if return_encoded:
            return encoded

        h, w = encoded.shape[-2:]

        encoded = rearrange(encoded, "b c h w -> b (h w) c")
        # quantized, _ = self.vq(encoded)
        quantized = self.vq(encoded)

        if self.use_vq_commit_loss:
            quantized, c_loss = self.get_vq_commitment_loss(quantized, encoded)

        quantized = rearrange(quantized, "b (h w) c -> b c h w", h=h, w=w)
        out = self.decoder(quantized)

        if not return_loss:
            return out

        # reconstruction loss and VQ commitment loss

        recon_loss = self.loss_fn(img, out)

        if self.use_vq_commit_loss:
            loss = recon_loss + c_loss
        else:
            loss = recon_loss

        if not return_recons:
            return loss, recon_loss

        return loss, recon_loss, out

In [ ]:
#| exports
class MaskedAutoencoderViT(nn.Module):
    """Masked Autoencoder with VisionTransformer backbone"""

    def __init__(
        self,
        img_size=256,
        patch_size=8,
        mask_ratio=0.75,
        in_chans=3,
        embed_dim=256,
        depth=24,
        num_heads=16,
        decoder_embed_dim=128,
        decoder_depth=8,
        decoder_num_heads=16,
        mlp_ratio=4.0,
        norm_layer=nn.LayerNorm,
        norm_pix_loss=False,
    ):
        super().__init__()

        self.mask_ratio = mask_ratio

        # --------------------------------------------------------------------------
        # MAE encoder specifics
        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, num_patches + 1, embed_dim), requires_grad=False
        )  # fixed sin-cos embedding

        self.blocks = nn.ModuleList(
            [
                Block(
                    embed_dim,
                    num_heads,
                    mlp_ratio,
                    qkv_bias=True,
                    # qk_scale=None,
                    norm_layer=norm_layer,
                )
                for i in range(depth)
            ]
        )
        self.norm = norm_layer(embed_dim)
        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(
            torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False
        )  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList(
            [
                Block(
                    decoder_embed_dim,
                    decoder_num_heads,
                    mlp_ratio,
                    qkv_bias=True,
                    # qk_scale=None,
                    norm_layer=norm_layer,
                )
                for i in range(decoder_depth)
            ]
        )

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(
            decoder_embed_dim, patch_size**2 * in_chans, bias=True
        )  # decoder to patch
        # --------------------------------------------------------------------------

        self.norm_pix_loss = norm_pix_loss

        self.initialize_weights()

    def initialize_weights(self):
        # initialization
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_2d_sincos_pos_embed(
            self.pos_embed.shape[-1],
            int(self.patch_embed.num_patches**0.5),
            cls_token=True,
        )
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_2d_sincos_pos_embed(
            self.decoder_pos_embed.shape[-1],
            int(self.patch_embed.num_patches**0.5),
            cls_token=True,
        )
        self.decoder_pos_embed.data.copy_(
            torch.from_numpy(decoder_pos_embed).float().unsqueeze(0)
        )

        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.patch_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=0.02)
        torch.nn.init.normal_(self.mask_token, std=0.02)

        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 *3)
        """
        p = self.patch_embed.patch_size[0]
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], 3, h, p, w, p))
        x = torch.einsum("nchpwq->nhwpqc", x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * 3))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size[0]
        h = w = int(x.shape[1] ** 0.5)
        assert h * w == x.shape[1]

        x = x.reshape(shape=(x.shape[0], h, w, p, p, 3))
        x = torch.einsum("nhwpqc->nchpwq", x)
        imgs = x.reshape(shape=(x.shape[0], 3, h * p, h * p))
        return imgs

    def random_masking(self, x):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - self.mask_ratio))

        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]

        # sort noise for each sample
        ids_shuffle = torch.argsort(
            noise, dim=1
        )  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward_encoder(self, x):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x, mask, ids_restore

    def forward_encoder_features(self, x):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x

    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(
            x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1
        )
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(
            x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2])
        )  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_decoder2(self, x):
        # embed tokens
        x = self.decoder_embed(x)

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove,
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.0e-6) ** 0.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss

    def forward(self, imgs):
        latent, mask, ids_restore = self.forward_encoder(imgs)
        pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
        loss = self.forward_loss(imgs, pred, mask)
        return loss, pred, mask

    def forward2(self, imgs):
        latent = self.forward_encoder_features(imgs)
        pred = self.forward_decoder2(latent)  # [N, L, p*p*3]
        return pred

In [ ]:
#| exports
def get_mae_model(cnfg):
    model = MaskedAutoencoderViT(
        patch_size=cnfg.patch_size,
        embed_dim=cnfg.encoder_embedding_dim,
        depth=cnfg.depth_enc,
        num_heads=cnfg.num_heads_enc,
        decoder_embed_dim=cnfg.decoder_embedding_dim,
        decoder_depth=cnfg.decoder_depth,
        decoder_num_heads=cnfg.decoder_num_heads,
        mlp_ratio=cnfg.mlp_ratio,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        img_size=cnfg.img_size,
        in_chans=cnfg.in_chans,
        mask_ratio=cnfg.mask_ratio,
    )

    return model

In [ ]:
#| exports
def get_embeddings_mae(model, samples):
    out = model.forward_encoder_features(samples)
    cls_token = torch.select(out, 1, 0)

    return cls_token

In [ ]:
#| exports
def get_vae_model(cnfg):
    vae = MyVQVAE(
        image_size=cnfg.img_size,
        num_layers=cnfg.num_layers,  # number of downsamples - ex. 256 / (2 ** 3) = (32 x 32 feature map)
        num_tokens=cnfg.num_tokens,  # number of visual tokens. in the paper, they used 8192, but could be smaller for downsized projects
        codebook_dim=cnfg.codebook_dim,  # codebook dimension
        hidden_dim=cnfg.hidden_dim,  # hidden dimension
        num_resnet_blocks=cnfg.num_resnet_blocks,  # number of resnet blocks
        use_vq_commit_loss=cnfg.use_vq_commit_loss,
    )

    return vae

In [ ]:
#| exports
def get_embeddings_vae(dvae_model, img):
    encoded = dvae_model.encoder(img)

    h, w = encoded.shape[-2:]

    encoded = rearrange(encoded, "b c h w -> b (h w) c")

    quantized = dvae_model.vq(encoded)

    if dvae_model.use_vq_commit_loss:
        quantized, c_loss = dvae_model.get_vq_commitment_loss(quantized, encoded)

    quantized = rearrange(quantized, "b (h w) c -> b c h w", h=h, w=w)

    return quantized